In [11]:
import sys
import os
from pathlib import Path
import torch
sys.path.append(os.getcwd())
sys.path.append(str(Path(os.getcwd()).parent))
from utils import *
from configuration import *
import numpy as np

if not os.path.exists('./evaluation/'):
    os.makedirs('./evaluation/')

In [12]:
args = parser.parse_args([])
args.task = 'temporal'
args.metric = 'acc'
args.SoftEva = True
args.DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
args = FormulateArgs(args)

In [13]:
test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset/')

In [18]:
datainfo

{'dataname': 'symbols',
 'N_feature': 1,
 'N_class': 6,
 'N_time': 64,
 'N_train': 300,
 'N_valid': 100,
 'N_test': 100}

In [23]:
import time
import pandas as pd

results = torch.zeros([15,10,3])
all_results = []

for seed in range(10):
    for ds in range(15):
        args.SEED = seed
        args.DATASET = ds

        valid_loader, datainfo = GetDataLoader(args, 'valid', path='../dataset/')
        test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset/')

        modelname = f"baseline_model_RNN_data_{args.DATASET:02d}_{datainfo['dataname']}_seed_{args.SEED:02d}.model"
        print(modelname)
        
        if os.path.isfile(f'./models/{modelname}'):                
            model = torch.load(f'./models/{modelname}', map_location=args.DEVICE)
            # model.UpdateArgs(args)

            SetSeed(args.SEED)

            evaluator = Evaluator(args).to(args.DEVICE)

            for x,y in valid_loader:
                X_valid, y_valid = x.to(args.DEVICE), y.to(args.DEVICE)
            # start_time = time.time()
            for x,y in test_loader:
                X_test, y_test = x.to(args.DEVICE), y.to(args.DEVICE)
            # end_time = time.time()
            # elapsed_time = end_time - start_time
            # average_time = elapsed_time / datainfo['N_test']
                
                

            acc_valid = evaluator(model, X_valid, y_valid)
            acc_test   = evaluator(model, X_test,  y_test)

            results[ds,seed,0] = acc_valid
            results[ds,seed,1] = acc_test
        else:
            results[ds,seed,:] = float('nan')
            
        inference_time = 0
        for x,y in test_loader:
            X_test, y_test = x.to(args.DEVICE), y.to(args.DEVICE)
            start_time = time.time()
            model(X_test[:1, :, :])
            end_time = time.time()
            inference_time = end_time - start_time
            break
        results[ds,seed,2] = inference_time
        
        temp_result = [datainfo['dataname'], seed, results[ds, seed, 0].item(), results[ds, seed, 1].item(), results[ds, seed, 2].item()]
        all_results.append(temp_result)

columns = ['dataset', 'seed', 'acc_valid', 'acc_test', 'average_time']
all_results.sort(key=lambda x: x[0])            
df = pd.DataFrame(all_results, columns=columns)
# Save the DataFrame to an Excel file
if not os.path.exists(f"./evaluation/"):
    os.makedirs(f"./evaluation/")
excel_filename = f"./evaluation/ELMAN_evaluation_results_analysis.xlsx"
df.to_excel(excel_filename, index=False)
print(f"Results have been saved to {excel_filename}")


baseline_model_RNN_data_00_cbf_seed_00.model
baseline_model_RNN_data_01_distalphalanxtw_seed_00.model
baseline_model_RNN_data_02_freezerregulartrain_seed_00.model
baseline_model_RNN_data_03_freezersmalltrain_seed_00.model
baseline_model_RNN_data_04_gunpointagespan_seed_00.model
baseline_model_RNN_data_05_gunpointmaleversusfemale_seed_00.model
baseline_model_RNN_data_06_gunpointoldversusyoung_seed_00.model
baseline_model_RNN_data_07_middlephalanxoutlineagegroup_seed_00.model
baseline_model_RNN_data_08_mixedshapesregulartrain_seed_00.model
baseline_model_RNN_data_09_powercons_seed_00.model
baseline_model_RNN_data_10_proximalphalanxoutlinecorrect_seed_00.model
baseline_model_RNN_data_11_selfregulationscp2_seed_00.model
baseline_model_RNN_data_12_slope_seed_00.model
baseline_model_RNN_data_13_smoothsubspace_seed_00.model
baseline_model_RNN_data_14_symbols_seed_00.model
baseline_model_RNN_data_00_cbf_seed_01.model
baseline_model_RNN_data_01_distalphalanxtw_seed_01.model
baseline_model_RNN_d

In [17]:
import pandas as pd
columns = ['dataset', 'seed', 'acc_valid', 'acc_test', 'average_time']
for ds in range(15):
    results[ds,:,0] = np.nanmean(results[ds,:,0])
    results[ds,:,1] = np.nanmean(results[ds,:,1])
    results[ds,:,2] = np.nanmean(results[ds,:,2])
df = pd.DataFrame(results, columns=columns)
# Save the DataFrame to an Excel file
excel_filename = f"./evaluation/evaluation_results_analysis.xlsx"
df.to_excel(excel_filename, index=False)
print(f"Results have been saved to {excel_filename}")

ValueError: Must pass 2-d input. shape=(15, 10, 3)

In [16]:
results

tensor([[[7.2000e-01, 6.0000e-01, 1.0487e-04],
         [6.6000e-01, 6.0000e-01, 8.2304e-05],
         [6.5000e-01, 6.0000e-01, 8.0578e-05],
         [7.2000e-01, 7.3000e-01, 7.2834e-05],
         [8.6000e-01, 8.6000e-01, 7.4317e-05],
         [7.1000e-01, 6.4000e-01, 8.5566e-05],
         [6.7000e-01, 6.1000e-01, 9.4399e-05],
         [6.7000e-01, 6.0000e-01, 8.5793e-05],
         [7.2000e-01, 7.5000e-01, 8.4291e-05],
         [7.9000e-01, 8.3000e-01, 7.3285e-05]],

        [[7.3000e-01, 7.7000e-01, 9.5570e-05],
         [7.3000e-01, 7.8000e-01, 9.3241e-05],
         [7.7000e-01, 7.9000e-01, 1.0339e-04],
         [7.1000e-01, 7.5000e-01, 8.3263e-05],
         [6.8000e-01, 7.3000e-01, 1.0799e-04],
         [7.2000e-01, 7.6000e-01, 9.5623e-05],
         [7.0000e-01, 8.1000e-01, 6.8510e-05],
         [7.5000e-01, 7.5000e-01, 1.0147e-04],
         [7.5000e-01, 8.0000e-01, 1.0111e-04],
         [7.4000e-01, 7.6000e-01, 9.0244e-05]],

        [[5.1000e-01, 4.7000e-01, 8.6837e-05],
         

In [5]:
N_selected_seeds = 3
re_temp = torch.nan_to_num(results, nan=-10000.)
values, indices = torch.topk(re_temp[:,:,0], N_selected_seeds, dim=1, largest=True, sorted=True)
selected_results = results.gather(1, indices.unsqueeze(-1).expand(-1, -1, 2))
selected_test_mean = selected_results[:,:,-1].mean(1)
selected_test_std = selected_results[:,:,-1].std(1)
selected_test_results = torch.cat([selected_test_mean.unsqueeze(-1), selected_test_std.unsqueeze(-1)], dim=1)

np.savetxt(f"./evaluation/top_{N_selected_seeds}_RNN_acc.txt", selected_test_results.numpy(), delimiter="\t", fmt='%.5f')